In [1]:
!pip install pulp
from pulp import *

In [2]:
# Creates a list of the products
months = ['000','JAN', 'FEB', 'MAR', 'APR', 'MAY', 'JUN']

# A dictionary of the unit cost per month
cost = {'000': 0,
    'JAN': 240, 
         'FEB': 250,
       'MAR': 265, 
         'APR': 285,
       'MAY': 280, 
         'JUN': 260}

# A dictionary of the demand per month
demand = {'000': 0,
          'JAN': 1000, 
         'FEB': 4500,
       'MAR': 6000, 
         'APR': 5500,
       'MAY': 3500, 
         'JUN': 4000}

# A dictionary of the min production per month
minOut = {'000': 0,
          'JAN': 2000, 
         'FEB': 1750,
       'MAR': 2000, 
         'APR': 2250,
       'MAY': 2000, 
         'JUN': 1750}

# A dictionary of the max production per month
maxOut = {'JAN': 4000, 
         'FEB': 3500,
       'MAR': 4000, 
         'APR': 4500,
       'MAY': 4000, 
         'JUN': 3500,
         '000': 0}

# Create the 'prob' variable to contain the problem data
prob = LpProblem("Upton Corporation Production Planning", LpMinimize)

# A dictionary of the output variables
o = LpVariable.dicts("output",months,0)

# A dictionary of the storage variables
s = LpVariable.dicts("storage",months,1500,6000)

# Create objective function
prob += lpSum([o[i]*cost[i] for i in months]) + lpSum([0.015*cost[months[i+1]]*0.5*(s[months[i+1]]+s[months[i]]) for i in range(6)]), "Total Cost"

#Enforce values of init period
prob += s["000"] == 2750, "initial    "

# Create constraints
for month in months:
    prob += o[month] >= minOut[month], "minOut{}  ".format(month)
    prob += o[month] <= maxOut[month], "maxOut{}  ".format(month)

for i in range(6):
    prob += s[months[i]]+o[months[i+1]]-demand[months[i+1]]==s[months[i+1]], "balance %s" % months[i]

In [3]:
print(prob)

Upton Corporation Production Planning:
MINIMIZE
285*output_APR + 250*output_FEB + 240*output_JAN + 260*output_JUN + 265*output_MAR + 280*output_MAY + 1.7999999999999998*storage_000 + 4.2375*storage_APR + 3.8625*storage_FEB + 3.675*storage_JAN + 1.95*storage_JUN + 4.125*storage_MAR + 4.05*storage_MAY + 0.0
SUBJECT TO
initial____: storage_000 = 2750

minOut000__: output_000 >= 0

maxOut000__: output_000 <= 0

minOutJAN__: output_JAN >= 2000

maxOutJAN__: output_JAN <= 4000

minOutFEB__: output_FEB >= 1750

maxOutFEB__: output_FEB <= 3500

minOutMAR__: output_MAR >= 2000

maxOutMAR__: output_MAR <= 4000

minOutAPR__: output_APR >= 2250

maxOutAPR__: output_APR <= 4500

minOutMAY__: output_MAY >= 2000

maxOutMAY__: output_MAY <= 4000

minOutJUN__: output_JUN >= 1750

maxOutJUN__: output_JUN <= 3500

balance_000: output_JAN + storage_000 - storage_JAN = 1000

balance_JAN: output_FEB - storage_FEB + storage_JAN = 4500

balance_FEB: output_MAR + storage_FEB - storage_MAR = 6000

balance_MAR: 

In [4]:
# Optimize

prob.solve()

# Print the status of the solved LP
print("Status = %s" % LpStatus[prob.status])

Status = Optimal


In [5]:
# Print the value of the variables at the optimum
for i in months:
    print("%s | s= %f | o= %f" % (i, s[i].varValue, o[i].varValue))

# Print the value of the objective
print("Objective = %f" % value(prob.objective))

000 | s= 2750.000000 | o= 0.000000
JAN | s= 5750.000000 | o= 4000.000000
FEB | s= 4750.000000 | o= 3500.000000
MAR | s= 2750.000000 | o= 4000.000000
APR | s= 1500.000000 | o= 4250.000000
MAY | s= 2000.000000 | o= 4000.000000
JUN | s= 1500.000000 | o= 3500.000000
Objective = 6209403.125000


In [6]:
print("Constraint  | Shadow Price | Slack")
for c in prob.constraints:
    print("%s | %f | %f"  %(c,prob.constraints[c].pi,prob.constraints[c].slack))

Constraint  | Shadow Price | Slack
initial____ | -271.537500 | -0.000000
minOut000__ | 0.000000 | -0.000000
maxOut000__ | 0.000000 | -0.000000
minOutJAN__ | 0.000000 | -2000.000000
maxOutJAN__ | -33.337500 | -0.000000
minOutFEB__ | 0.000000 | -1750.000000
maxOutFEB__ | -27.012500 | -0.000000
minOutMAR__ | 0.000000 | -2000.000000
maxOutMAR__ | -15.875000 | -0.000000
minOutAPR__ | 0.000000 | -2000.000000
maxOutAPR__ | 0.000000 | 250.000000
minOutMAY__ | 0.000000 | -2000.000000
maxOutMAY__ | 0.000000 | -0.000000
minOutJUN__ | 0.000000 | -1750.000000
maxOutJUN__ | -24.050000 | -0.000000
balance_000 | 273.337500 | -0.000000
balance_JAN | 277.012500 | -0.000000
balance_FEB | 280.875000 | -0.000000
balance_MAR | 285.000000 | -0.000000
balance_APR | 280.000000 | -0.000000
balance_MAY | 284.050000 | -0.000000
